In [126]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import glob

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/tabea/Documents/UrbanMobility/src')
from models import transformer as transformer
from models import random_forest_model as random_forest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
journey_16 = pd.read_csv('../data/processed/aggregated_journey_data/agg_journey_data_2016_with_0_demand.csv', index_col=0)
journey_17 = pd.read_csv('../data/processed/aggregated_journey_data/agg_journey_data_2017_with_0_demand.csv', index_col=0)
journey_18 = pd.read_csv('../data/processed/aggregated_journey_data/agg_journey_data_2018_with_0_demand.csv', index_col=0)
journey_19 = pd.read_csv('../data/processed/aggregated_journey_data/agg_journey_data_2019_with_0_demand.csv', index_col=0)

In [145]:
journey_train = pd.concat([journey_16, journey_17, journey_18])
journey_test = journey_19.copy()

In [146]:
x_train, y_train, x_test, y_test = random_forest.train_test_split(journey_train, journey_test)

In [147]:
x_train.head()

,day_of_week,hour,is_weekend,part_of_day,month,season,bank_holiday,day_of_month,day_of_year,temp,...,start_borough_Hackney,start_borough_Hammersmith and Fulham,start_borough_Islington,start_borough_Kensington and Chelsea,start_borough_Lambeth,start_borough_Newham,start_borough_Southwark,start_borough_Tower Hamlets,start_borough_Wandsworth,start_borough_Westminster
0,4.0,0.0,0.0,5.0,1.0,4.0,1.0,1.0,1.0,5.1,...,0,0,0,0,0,0,0,0,0,0
1,4.0,0.0,0.0,5.0,1.0,4.0,1.0,1.0,1.0,5.1,...,0,0,0,0,0,0,0,0,0,0
2,4.0,0.0,0.0,5.0,1.0,4.0,1.0,1.0,1.0,5.1,...,1,0,0,0,0,0,0,0,0,0
3,4.0,0.0,0.0,5.0,1.0,4.0,1.0,1.0,1.0,5.1,...,0,1,0,0,0,0,0,0,0,0
4,4.0,0.0,0.0,5.0,1.0,4.0,1.0,1.0,1.0,5.1,...,0,0,1,0,0,0,0,0,0,0


In [157]:
n_boroughs = 12

# reshape input to be 3D [samples, timesteps, features]
x_train_reshaped = x_train.values.reshape((x_train.shape[0] // n_boroughs, n_boroughs, x_train.shape[1]))
x_test_reshaped = x_test.values.reshape((x_test.shape[0] // n_boroughs, n_boroughs, x_test.shape[1]))

# We don't reshape y, since it should be a 1D array
y_train_reshaped = y_train.values.reshape(-1, n_boroughs).mean(axis=1)
y_test_reshaped = y_test.values.reshape(-1, n_boroughs).mean(axis=1)


In [159]:
print(x_train_reshaped.shape, y_train_reshaped.shape)

(26229, 12, 104) (26229,)


In [164]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *

model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 10, return_sequences = True, input_shape = (x_train_reshaped.shape[1], x_train_reshaped.shape[2])))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 10, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 10, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 10))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 12, 10)            4600      
                                                                 
 dropout_28 (Dropout)        (None, 12, 10)            0         
                                                                 
 lstm_5 (LSTM)               (None, 12, 10)            840       
                                                                 
 dropout_29 (Dropout)        (None, 12, 10)            0         
                                                                 
 lstm_6 (LSTM)               (None, 12, 10)            840       
                                                                 
 dropout_30 (Dropout)        (None, 12, 10)            0         
                                                                 
 lstm_7 (LSTM)               (None, 10)               

In [165]:
from keras.callbacks import EarlyStopping

model.compile(optimizer = 'adam', loss = 'mean_squared_error')
# Fitting the model to the Training set
history = model.fit(x_train_reshaped, y_train_reshaped, epochs = 10, batch_size = 32, validation_split= 0.2,
                    callbacks = EarlyStopping(monitor='val_loss', patience = 5))
     

Epoch 1/10
656/656 [==============================] - 21s 21ms/step - loss: 18941.0742 - val_loss: 16562.0977
Epoch 2/10
656/656 [==============================] - 12s 18ms/step - loss: 17919.1797 - val_loss: 15681.5596
Epoch 3/10
656/656 [==============================] - 12s 18ms/step - loss: 17067.1094 - val_loss: 14992.7529
Epoch 4/10
656/656 [==============================] - 12s 18ms/step - loss: 16382.2354 - val_loss: 14417.5723
Epoch 5/10
656/656 [==============================] - 12s 18ms/step - loss: 15820.4736 - val_loss: 13938.6123
Epoch 6/10
485/656 [=====================>........] - ETA: 3s - loss: 15311.2441

KeyboardInterrupt: 

In [160]:
## Testing the Transformer
tr = transformer.Transformer()
tr.train(x_train_reshaped,y_train_reshaped)


Epoch 1/3
820/820 [==============================] - 20s 21ms/step - loss: 9200.8945
Epoch 2/3
820/820 [==============================] - 19s 23ms/step - loss: 8932.6064
Epoch 3/3
820/820 [==============================] - 23s 28ms/step - loss: 8936.7354


In [161]:
from sklearn.metrics import mean_squared_error

# Predict outputs for the test set
y_pred = tr.predict(x_test_reshaped)

# Calculate the Mean Squared Error of the predictions
mse = mean_squared_error(y_test_reshaped, y_pred)

print(f'The Mean Squared Error of the predictions is {mse}')

273/273 [==============================] - 2s 7ms/step
The Mean Squared Error of the predictions is 8858.326241111405
